In [1]:
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_lg-2.2.0/en_core_web_lg-2.2.0.tar.gz

     |████████████████████████████████| 827.9 MB 3.9 kB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.0-py3-none-any.whl size=829187827 sha256=0fdd2983750c6b229da57c8df51ca2c943dbae05c4b758cff0d0e8af3d2ef00b
  Stored in directory: /root/.cache/pip/wheels/4b/6e/87/8b129988ba4d66c064415e09db36d5945b638aabcaf8f41765
Successfully built en-core-web-lg


In [1]:
!pip install bert-tensorflow

     |████████████████████████████████| 64 kB 1.8 MB/s 


In [2]:
#%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
!ln -s /content/gdrive/MyDrive/ /mydrive
!ls /mydrive

 2021-online-language-offline-impact-main
 An_Xception_Based_Convolutional_Neural_Network_for_Scene_Image_Classification_with_Transfer_Learning.pdf
'Application for Lease- Form for Email (Applicant) (1).gdoc'
'Application for Lease- Form for Email (Applicant).gdoc'
'Application for Lease- Form for Email (Applicant).pdf'
'Colab Notebooks'
'CV -Xizhi Wu.pdf'
'FOW Documents'
 MyDrive
'NT15 LEASE 2021.gdoc'
'NT15 LEASE 2021.pdf'
 Xizhi_Wu_UnderGrad_Transcript.pdf


In [4]:
%cd gdrive/MyDrive/2021-online-language-offline-impact-main

/content/gdrive/MyDrive/2021-online-language-offline-impact-main


In [5]:
!ls

 a0529.csv	 heatmap530.ipynb
 a0530.csv	 heatmap531.ipynb
 a0531.csv	 Heat_Map_Online_language.ipynb
 a0601.csv	 __MACOSX
 a0602.csv	 non_violant1.csv
 csv		 non_violant2.csv
 csv.zip	 violant.csv
 dataframe_csv	'新建 Microsoft PowerPoint 演示文稿.pptx'
 Github


In [6]:
import os
import collections
import ast
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime, timedelta
import spacy
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
from bert import tokenization
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import matplotlib.pyplot as plt

In [7]:
ct_path_603 = 'csv/US06_03.csv'

In [8]:
ct = pd.read_csv(ct_path_603)

In [9]:
ct.shape[1]

47

In [10]:
columns1=['platformId', 'platform', 'date', 'updated', 'type1', 'title', 'caption', 'description', 'message','expandedLinks', 'link', 'postUrl', 'subscriberCount', 'score', 'media_type', 'media_numl', 'actual_likeCount', 'actual_shareCount', 'actual_commentCount', 'actual_loveCount', 'actual_wowCount', 'actual_hahaCount', 'actual_sadCount', 'actual_angryCount', 'actual_thankfulCount', 'actual_careCount', 'expected_likeCount', 'expected_shareCount', 'expected_commentCount', 'expected_loveCount', 'expected_wowCount', 'expected_hahaCount', 'expected_sadCount', 'expected_angryCount', 'expected_thankfulCount', 'expected_careCount', 'account_id', 'account_name', 'account_handle', 'account_profileImage', 'account_accountType', 'account_pageAdminTopCountry', 'account_pageDescription', 'account_verified', 'account_pageCreatedDate', 'languageCode', 'legacyId', 'post_id']
columns2=['platformId', 'platform', 'date', 'updated', 'type1', 'title', 'caption', 'description', 'message', 'expandedLinks', 'link', 'postUrl', 'subscriberCount', 'score', 'media', 'actual_likeCount', 'actual_shareCount', 'actual_commentCount', 'actual_loveCount', 'actual_wowCount', 'actual_hahaCount', 'actual_sadCount', 'actual_angryCount', 'actual_thankfulCount', 'actual_careCount', 'expected_likeCount', 'expected_shareCount', 'expected_commentCount', 'expected_loveCount', 'expected_wowCount', 'expected_hahaCount', 'expected_sadCount', 'expected_angryCount', 'expected_thankfulCount', 'expected_careCount', 'account_id', 'account_name', 'account_handle', 'account_profileImage', 'account_accountType', 'account_pageAdminTopCountry', 'account_pageDescription', 'account_verified', 'account_pageCreatedDate', 'languageCode', 'legacyId', 'post_id']

In [11]:
#len(columns1)
len(columns2)

47

In [12]:
ct.columns=columns2

In [13]:
ct

,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id
0,122692991088722_3333978479960141,Facebook,2020-06-03 22:31:01,2021-07-23 06:19:04,link,'Just Mercy' Available For Free To Educate Peo...,wyht.iheart.com,"In the wake of George Floyd's murder, Warner B...","In the wake of George Floyd's murder, Warner B...",1.0,http://ihe.art/6NYcmZP,https://www.facebook.com/122692991088722/posts...,2136,-22.000000,1.0,0,0,0,0,0,0,0,0,0,0,1,2,1,1,1,2,2,1,0,1,1573932,"Y-105 More Music, More Variety",Y105Mansfield,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,"Y-105 is Mid-Ohio's station with More Music, M...",True,2010-05-17 15:57:20,en,0,1573932|3333978479960141
1,141811187459_10158302375452460,Facebook,2020-06-03 22:31:00,2021-06-18 17:25:33,link,'Just Mercy' Available For Free To Educate Peo...,power99.iheart.com,"In the wake of George Floyd's murder, Warner B...","In the wake of George Floyd's murder, Warner B...",1.0,http://ihe.art/pIAnLtI,https://www.facebook.com/141811187459/posts/10...,263601,2.333333,1.0,26,22,0,15,0,0,0,0,0,1,3,2,3,3,3,4,4,5,0,2,39724,Power 99FM Philly,Power99Philly,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_page,US,We are Philly's Power 99! Check us out at Powe...,True,2009-09-24 12:45:53,en,0,39724|10158302375452460
2,216833661707402_3142093555848050,Facebook,2020-06-03 22:31:00,2021-07-10 10:16:53,link,Four officers charged in George Floyd's death,hennepinattorney.org,The other three former Minneapolis police offi...,"Standing together at a news conference, Minnes...",1.0,https://www.hennepinattorney.org/news/news/202...,https://www.facebook.com/216833661707402/posts...,2980,1.578947,1.0,14,7,8,1,0,0,0,0,0,1,3,2,2,2,2,1,4,2,0,1,1831545,Hennepin County Attorney's Office,HennepinAttorney,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,"We prosecute crimes, advocate for victims and ...",False,2011-09-06 16:00:14,en,0,1831545|3142093555848050
3,139637712766018_3223165654413193,Facebook,2020-06-03 22:31:00,2021-07-30 20:39:10,link,'Just Mercy' Available For Free To Educate Peo...,star1029.iheart.com,"In the wake of George Floyd's murder, Warner B...","In the wake of George Floyd's murder, Warner B...",1.0,http://ihe.art/gItRAjD,https://www.facebook.com/139637712766018/posts...,4027,-20.000000,1.0,0,0,0,0,0,0,0,0,0,0,1,2,1,1,1,1,1,2,0,1,478045,Star 102.9,Star1029,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_page,US,"Star 102.9/107.7, The Four Corners' Hit Music ...",True,2011-02-07 17:56:15,en,0,478045|3223165654413193
4,50283913143_10157242396648144,Facebook,2020-06-03 22:31:00,2021-07-10 14:57:11,link,'Just Mercy' Available For Free To Educate Peo...,jamn945.iheart.com,"In the wake of George Floyd's murder, Warner B...","In the wake of George Floyd's murder, Warner B...",1.0,http://ihe.art/9cbDv3O,https://www.facebook.com/50283913143/posts/101...,142626,2.750000,1.0,50,19,1,7,0,0,0,0,0,0,3,2,3,3,3,2,5,7,0,1,39129,JAM'N 94.5,jamn945,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Boston's #1 For Hip Hop & The Best Throwbacks\...,True,2009-02-12 19:00:57,en,0,39129|10157242396648144
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29104,71966175060_1

In [14]:
ct['message'] = ct['message'].fillna('')
ct['description'] = ct['description'].fillna('')
ct['title'] = ct['title'].fillna('')

In [15]:
nlp = spacy.load('en_core_web_lg', disable=["tok2vec", "tagger", "parser", "attribute_ruler", "lemmatizer"])

In [16]:
def ner(txt):
    ner = dict()

    doc = nlp(txt)
    for ent in doc.ents:
        if ent.label_ == 'GPE':
            if ent.text not in ner:
                ner[ent.text] = 1
            else:
                ner[ent.text] += 1
    return ner

In [17]:
def ner_inline(doc):
    ner = dict()

    for ent in doc.ents:
        if ent.label_ == 'GPE':
            if ent.text not in ner:
                ner[ent.text] = 1
            else:
                ner[ent.text] += 1
    return ner

In [18]:
def ner_pipe(texts):
    batch = []
    for doc in nlp.pipe(texts, batch_size=200, n_threads=16):
        batch.append(ner_inline(doc))
    return batch

In [19]:
ct['NER-msg'] = ner_pipe(ct['message'])

In [20]:
ct['NER-msg'] = ct['message'].apply(ner)

In [21]:
ct['NER-desc'] = ct['description'].apply(ner)

In [22]:
ct['NER-title'] = ct['title'].apply(ner)

In [23]:
ct[['NER-msg', 'NER-desc']].head()

,NER-msg,NER-desc
0,{},{}
1,{},{}
2,"{'Minnesota': 1, 'Hennepin County': 2}","{'Minneapolis': 1, 'Minnesota': 1, 'Hennepin C..."
3,{},{}
4,{},{}


In [24]:
ct.to_csv('a0603.csv')

In [25]:
ct=pd.read_csv('a0603.csv')

In [26]:
ct['NER-msg'] = ct['NER-msg'].apply(ast.literal_eval)
ct['NER-desc'] = ct['NER-desc'].apply(ast.literal_eval)
ct['NER-title'] = ct['NER-title'].apply(ast.literal_eval)

In [27]:
ct_ner = ct[((ct['NER-msg'] != {}) | (ct['NER-desc'] != {}) | (ct['NER-title'] != {}))]

In [28]:
ct_ner[['NER-msg', 'NER-desc', 'NER-title']].head()

,NER-msg,NER-desc,NER-title
2,"{'Minnesota': 1, 'Hennepin County': 2}","{'Minneapolis': 1, 'Minnesota': 1, 'Hennepin C...",{}
10,{},{'U.S.': 1},{}
11,"{'CEDAR RAPIDS': 1, 'Iowa': 2, 'Minneapolis': 1}","{'Minneapolis': 1, 'Iowa': 1, 'the United Stat...",{'Iowa': 1}
12,{},"{'Minneapolis': 1, 'Iowa': 1, 'the United Stat...",{'Iowa': 1}
14,{},{'WASHINGTON': 1},{}


In [29]:
ct_ner['NER-msg'] = ct_ner['NER-msg'].apply(lambda x: list(set(x.keys())))
ct_ner['NER-desc'] = ct_ner['NER-desc'].apply(lambda x: list(set(x.keys())))
ct_ner['NER-title'] = ct_ner['NER-title'].apply(lambda x: list(set(x.keys())))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [30]:
ct_ner['NER-msg']

2                             [Hennepin County, Minnesota]
10                                                      []
11                       [Minneapolis, CEDAR RAPIDS, Iowa]
12                                                      []
14                                                      []
                               ...                        
29101                                        [Minneapolis]
29102                                            [Germany]
29103                                                   []
29105    [North Minneapolis, MN, Jordan, Minnesota, MN ...
29108                                                   []
Name: NER-msg, Length: 19711, dtype: object

In [31]:
ct_ner['NER'] = ct_ner['NER-msg'] + ct_ner['NER-desc'] + ct_ner['NER-title']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [32]:
ct_ner['NER']

2        [Hennepin County, Minnesota, Minneapolis, Minn...
10                                                  [U.S.]
11       [Minneapolis, CEDAR RAPIDS, Iowa, Minneapolis,...
12            [Minneapolis, the United States, Iowa, Iowa]
14                                            [WASHINGTON]
                               ...                        
29101          [Minneapolis, Minneapolis, Minn., St. Paul]
29102                                   [Germany, Germany]
29103                       [the United States of America]
29105    [North Minneapolis, MN, Jordan, Minnesota, MN ...
29108                                         [WASHINGTON]
Name: NER, Length: 19711, dtype: object

In [33]:
ct_ner['NER'] = ct_ner['NER'].apply(set)
ct_ner['NER'] = ct_ner['NER'].apply(list)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [34]:
ct_ner['NER'] = ct_ner['NER'].apply(lambda x: [s.lower() for s in x])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [35]:
ner_lst = ct_ner['NER'].values

In [36]:
ner_lst

array([list(['minneapolis', 'hennepin county', 'minnesota']),
       list(['u.s.']),
       list(['minneapolis', 'the united states', 'cedar rapids', 'iowa']),
       ..., list(['the united states of america']),
       list(['north minneapolis', 'mn', 'jordan', 'minnesota', 'mn 55352', 'south minneapolis', 'grove']),
       list(['washington'])], dtype=object)

In [37]:
m=0
list=[]
for n in ner_lst:
  for i in range(len(n)):
    list.append(ner_lst[m][i])
  m+=1

In [38]:
list

['minneapolis',
 'hennepin county',
 'minnesota',
 'u.s.',
 'minneapolis',
 'the united states',
 'cedar rapids',
 'iowa',
 'minneapolis',
 'the united states',
 'iowa',
 'washington',
 'louisville',
 'stockton',
 'ephesus',
 'stockton',
 'america',
 'philando castile',
 'the united states',
 'america',
 'des moines',
 'minneapolis',
 'statesville',
 'america',
 'nc',
 'atlanta',
 'minneapolis',
 'atlanta',
 'u.s.',
 'minnesota',
 'https://www.sbdiocese.org/docs/2020/05-29-2020_statement_on_g_floyd.pdf',
 'minneapolis',
 'north idaho',
 'spokane',
 'u.s.',
 'london',
 'us',
 'los angeles',
 'america',
 'massachusetts',
 'america',
 'st. louis',
 'missouri',
 'ferguson',
 'the united states',
 'minneapolis',
 'u.s.',
 'washington',
 'minneapolis',
 'minneapolis',
 'houston',
 'n.y.',
 'buffalo',
 'the city of buffalo',
 'minneapolis',
 'the united states',
 'iowa',
 'iowa',
 'iowa',
 'iowa',
 'minneapolis',
 'fort myers',
 'minneapolis',
 'minneapolis',
 'u.s.',
 'minneapolis',
 'usa',


In [39]:
c = collections.Counter()

In [40]:
dict_a=collections.Counter(list).most_common()

In [41]:
dict_a

[('minneapolis', 8294),
 ('minnesota', 3210),
 ('america', 1824),
 ('u.s.', 1263),
 ('us', 1073),
 ('the united states', 993),
 ('washington', 817),
 ('houston', 632),
 ('new york city', 457),
 ('new york', 420),
 ('los angeles', 388),
 ('chicago', 349),
 ('texas', 298),
 ('d.c.', 251),
 ('florida', 234),
 ('atlanta', 229),
 ('california', 204),
 ('portland', 202),
 ('philadelphia', 201),
 ('new jersey', 197),
 ('brooklyn', 191),
 ('nj', 191),
 ('alabama', 175),
 ('london', 172),
 ('georgia', 169),
 ('louisville', 165),
 ('dc', 163),
 ('seattle', 161),
 ('maryland', 158),
 ('virginia', 151),
 ('boston', 150),
 ('mn', 146),
 ('manhattan', 145),
 ('paris', 140),
 ('nyc', 136),
 ('north carolina', 136),
 ('denver', 132),
 ('usa', 130),
 ('dallas', 125),
 ('st. louis', 122),
 ('oakland', 122),
 ('missouri', 117),
 ('las vegas', 116),
 ('detroit', 112),
 ('iowa', 111),
 ('milwaukee', 105),
 ('st. paul', 102),
 ('michigan', 101),
 ('miami', 95),
 ('kentucky', 94),
 ('wisconsin', 93),
 ('holl

In [42]:
dataframe603=pd.DataFrame(dict_a)

In [43]:
len(dict_a)

3085

In [44]:
dataframe603.to_csv('dataframe0603.csv')